<a href="https://colab.research.google.com/github/RajidiSahithi/Session_10/blob/main/S10_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>